In [1]:
import requests
import pandas as pd
import numpy as np
import scr ## scr.py custom function ## scr.parse("")

import contractions
from bs4 import BeautifulSoup
import lxml

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")

import warnings
warnings.filterwarnings('ignore')

from random import randint
from time import sleep

#### Conditions setup

In [2]:
max_scrape = 2000 # while loop max
num_reviews = 20 # you can adjust this number on how many reviews you which to scrape

In [3]:
all_reviews_df = pd.DataFrame(columns = ['rating', 'rating_title', 
'rating_description','rating_pros', 'rating_cons'])

In [4]:
def parse(full_url):
    
    page_content = BeautifulSoup(full_url.content, 'lxml')
    containers = page_content.findAll('div', 
                 {'data-tn-section':'reviews'})
    
    global df, df1
    
    df = pd.DataFrame(columns = 
         ['rating', 'rating_title',  'rating_description',
                         'rating_pros', 'rating_cons'])
    
    for item in containers:
        #RATING
        try:
            rating = item.find('button', 
                     {'class': 'css-1c33izo e1wnkr790'}).text.replace('\n', '')

        except:
            rating = None
                     
        # TITLE             
        try:
            rating_title = item.find('h2', 
                           {'data-testid': 'title'}).text.replace('\n', '')

        except:
            rating_title = None

        #DESCRIPTION 
        try:
            rating_description = item.find('div', 
                                 {'data-tn-component': 'reviewDescription'}).text.replace('\n', '.')

        except:
            rating_description = None
  
        #PROS
        try:
            rating_pros = item.find('div', 
                          {'class': 'css-1z0411s e1wnkr790'}).text.replace('\n', '')

                        
        except:
            rating_pros = None

        #CONS
        try:
            rating_cons = page_content.findAll('class', 
                 {'css-82l4gy eu4oa1w0'})

            for i, j in zip(rating_cons, range(1, 3)):
                if j == 1:
                    pass
                else:
                    output = i
                          
        except:
            
            rating_cons = None
        
        
        df = df.append({'rating': rating, 
             'rating_title': rating_title, 
             'rating_description': rating_description,
             'rating_pros': rating_pros,
             'rating_cons': rating_cons}, ignore_index=True)

    return df

#### Scrape

In [5]:
base_url = 'https://www.indeed.com/cmp/Google/reviews?fcountry=ALL&start='

In [6]:
while num_reviews < max_scrape:  
    
    full_url = base_url + str(num_reviews)
    
    get_url = requests.get(full_url, timeout=30)  
        
    partial_reviews_df = parse(get_url)

    all_reviews_df = all_reviews_df.append(
                     partial_reviews_df, ignore_index=True) 
    
    num_reviews += 20
    
    sleep(randint(1, 10))

In [7]:
all_reviews_df.head(5)

,rating,rating_title,rating_description,rating_pros,rating_cons
0,5.0,amazing,Really loved it. such a great time. entire tea...,None,[]
1,5.0,"Loved this job, the team was great to work wit...",Absolutely loved working with Google. They wer...,Basically your own boss. Make your own hours. ...,[]
2,4.0,Great but stressful,Great great place to work at. But extreme hou...,Salary Benefits Work collaboration,[]
3,5.0,Fun,Excellent place it work Excellent employees Ex...,great environment,[]
4,5.0,Productive and fun,"Productive and fun, Worked here since I was a ...",Free lunches,[]


In [8]:
all_reviews_df.shape

(2079, 5)

In [9]:
all_reviews_df.to_csv('data/all_reviews_df.csv')